<center><h2> Image Classification for Car Models </h2></center>
<hr>

### Connecting to Google Drive
- Installing PyDrive and mount Google Drive to Google Colab
- Change the current directiory to current drive to access own drive

In [0]:
# Install PyDrive
!pip install PyDrive

In [0]:
# Mounting Google Drive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Change Directory to My Drive
%cd drive
%cd 'My Drive'

### Data Pre-processing
- Organise the images in a folder of their respective labels (classes)
- Split into train and validation folders and copy the respective images over

In [0]:
# Unzip the folder of training images
!unzip train

In [0]:
import cv2
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Retrieve all the images in train folder and append them to a list
train = []
train_images = glob(os.path.join('data/train', "*.jpg"))
for image in train_images:
    x = plt.imread(image)
    train.append(x)

In [0]:
# Retrieve the labels from the excel file and store it in a list
test_labels = []
labels_df = pd.ExcelFile('D:\Competitions\Grab Competition - Computer Vision\\test_labels.xlsx').parse('Sheet1')
test_labels.append(labels_df['class'].values)
test_labels = test_labels[0].tolist()

In [0]:
# Organise the training images in a folder of their respective labels
final_train_labels = []
counter = 0
total_train = []
total_train.append(train_df['fname'].values)
total_train = total_train[0].tolist()
for image in train_images:
    counter = 0
    for i in total_train:
        i = 'data/train/' + i
        if i == image:
            final_train_labels.append(train_labels[counter])
            break
        counter += 1
print(final_train_labels)

In [0]:
# Organise the test images in a folder of their respective labels
destination = "D:\\Competitions\\Grab Competition - Computer Vision\\test"
os.makedirs(destination)
counter = 0
classes_reg = []
for image in test_images:
    class_number = test_labels[counter]
    dest = destination
    dest += "\\" + str(class_number)
    if class_number not in classes_reg:
        os.makedirs(dest)
        classes_reg.append(class_number)
    shutil.copy(image,dest)
    counter += 1

In [0]:
# Split the images into training and validation set of 80:20 ratio
df = pd.DataFrame({'x' : train_images, 'y': train_labels})
X_train, X_test, y_train, y_test = train_test_split(df, train_labels, test_size=0.2)
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))
X_train = list(X_train.loc[:, 'x'])
X_test = list(X_test.loc[:, 'x'])

# Move the images into the respective directory
train = "D:\Competitions\Grab Competition - Computer Vision\data1\\train"
val = "D:\Competitions\Grab Competition - Computer Vision\data1\\val"
os.makedirs(train)
os.makedirs(val)
for image in X_train:
    shutil.copy(image, train)
    
for test in X_test:
    shutil.copy(test, val)

In [0]:
# Convert into numpy array for better performance and suitability for input for model
val = np.asarray(val)
np.save("test", val)

### Data Augementation
- To multiply training images to reduce overfitting with the use of augmentation
- Instance Segmentation with pre-trained Mask RCNN model
- Crop instances and paste it over self-selected background images
- Filter the images and delete those unneeded

In [0]:
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Downlaod Instance Segmentation Model to idenitify the mask
!git clone https://www.github.com/matterport/Mask_RCNN.git

# Change the directory to the Mask RCNN model
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))
os.chdir('Mask_RCNN')
ROOT_DIR = os.getcwd()
sys.path.append(os.path.join(ROOT_DIR, "mrcnn/"))

In [ ]:
import PIL
import mrcnn
import mrcnn.model as modellib
import tensorflow as tf
import matplotlib.image as mpimg
from PIL import Image
from google.colab import files
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import visualize
from mrcnn.model import log
from mrcnn.visualize import display_images

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
import coco
# Instantiate a coco object
config = coco.CocoConfig()
COCO_DIR = '/content/drive/My Drive/data/train'

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [0]:
# Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir='content/Mask_RCNN/mask_rcnn_coco.h5',
                              config=config)

# Set weights file path
if config.NAME == "coco":
    weights_path = COCO_MODEL_PATH

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [0]:
# Read the chosen background images and load them into a list
background_images = []
background_folder = glob(os.path.join('/content/drive/My Drive/background', '*.jpg'))
  
for p in background_folder:
  background_images.append(plt.imread(str(p)))

In [0]:
# For every class and for every image, randomly select a background image
# Crop out the mask of the image using the Mask RCNN model and paste the car instance segmented onto the background
for j in range(0, 197):
  images = []
  os.chdir('/content/drive/My Drive/data/train/' + str(j))
  folder = glob(os.path.join('/content/drive/My Drive/data/train/' + str(j), '*.jpg'))

  for i in folder:
    images.append(plt.imread(str(i)))

  counter = 0
  for k in images:
    try:
      results = model.detect([k], verbose=1)
      mask = results[0]['masks']

      temp_image = np.copy(k)
      for m in range(k.shape[0]):
        for n in range(k.shape[1]):
          if mask[m][n][0] != False:
            continue
          else:
            for l in range(3):
              temp_image[m][n][l] = 0

      index = random.randint(0, 19)
      background = background_images[index]

      edited_back = cv2.resize(background, (k.shape[1], k.shape[0]))
      new_image = np.copy(edited_back)

      for q in range(k.shape[0]):
        for w in range(k.shape[1]):
          for e in range(3):
            if temp_image[q][w][e] != 0:
              new_image[q][w][e] = temp_image[q][w][e]

      counter = counter + 1
      im = Image.fromarray(new_image)
      im.save('new' + str(j) + "_" + str(counter) + '.png')
    except:
      continue

90
Processing 1 images
image                    shape: (225, 300, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (299, 601, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (225, 300, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  15

In [0]:
# View the augmented images of each class to filter the required ones out
os.chdir('/content/drive/My Drive/data/train/168')

fig = plt.gcf()
fig.set_size_inches(15, 15)
images_paths = os.listdir('/content/drive/My Drive/data/train/168')

nrows = 4
ncols = 4
pic_index = 36

pic_index += 16
pics = [os.path.join(fname) for fname in images_paths[pic_index-16:pic_index]]

for i, img_path in enumerate(pics):
    sp = plt.subplot(nrows, ncols, i + 1)
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

### Training Neural Networks
- Prepare the image generator and the selected pre-trained models for transfer learning
- Train the model through selected epochs
- Plot the learning curve and the validation loss to avoid overfitting

In [0]:
import keras
from efficientnet import *
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from tensorflow.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

# Create the directory for respective folders
train_folder = os.path.join('/content/drive/My Drive/data/train')
val_folder = os.path.join('/content/drive/My Drive/data/val')
test_folder = os.path.join('/content/drive/My Drive/test')

# Batch size
bs_train = 6515
bs_val = 1000
bs_test = 6000

# All images will be resized to this value
image_size = (300, 300)

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range= [0.5,1.5],
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images using train_datagen generator
print("Preparing generator for train dataset")
train_generator = train_datagen.flow_from_directory(
    directory= train_folder, 
    target_size=image_size,
    batch_size=bs_train,
    class_mode='categorical')

# Flow validation images using val_datagen generator
print("Preparing generator for validation dataset")
val_generator = val_datagen.flow_from_directory(
    directory= val_folder, 
    target_size=image_size,
    batch_size=bs_val,
    class_mode='categorical')

# Flow validation images using test_datagen generator
print("Preparing generator for test dataset")
test_generator = test_datagen.flow_from_directory(
    directory= test_folder, 
    target_size=image_size, 
    batch_size=bs_test,
    class_mode='categorical')

In [0]:
# Install efficientnet
!pip install -U git+https://github.com/qubvel/efficientnet

In [0]:
# Build Model using pre-trained weights
base_model = EfficientNetB5(weights = 'imagenet', include_top = False, input_shape = (300, 300, 3))
for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(196, activation = 'softmax'))

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer=tensorflow.keras.optimizers.RMSprop(lr = 0.0001), metrics=['accuracy'])

In [0]:
# Check the layers of the model
model.summary()

In [0]:
# Training Models
history = model.fit_generator(
        train_generator, 
        steps_per_epoch=train_generator.samples // bs_train + 1,
        epochs=1,
        validation_data=val_generator, 
        validation_steps=val_generator.samples // bs_val + 1
)

In [0]:
# Saving model in h5 format
model.save('efficient.h5')

In [ ]:
# Building of second model
base_model = InceptionResnet(weights = 'imagenet', include_top = False, input_shape = (300, 300, 3))
for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(196, activation = 'softmax'))

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer=tensorflow.keras.optimizers.RMSprop(lr = 0.0001), metrics=['accuracy'])

In [ ]:
# Training Model
history = model.fit_generator(
        train_generator, 
        steps_per_epoch=train_generator.samples // bs_train + 1,
        epochs=1,
        validation_data=val_generator, 
        validation_steps=val_generator.samples // bs_val + 1
)

In [ ]:
# Saving model in h5 format
model2.save('inceptionresnetv2_modelv6.h5')

In [0]:
# load model
from efficientnet import load_model

model_path = 'inceptionresnetv2_modelv6.h5'
model1 = load_model(model_path)

In [0]:
# Plot validation loss graph
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [0]:
# Plot learning curves to check for overfitting
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

### Ensemble Learning
- Using Grid Search to find the best parameters for ensemble learning
- The weighted combinations from models will provide the predicted outputs of best accuracy

In [0]:
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product
from sklearn.ensemble import VotingClassifier
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

# Make an ensemble prediction for multi-class classification
def ensemble_predictions(members, weights, testX):
	# make predictions
  yhats = [model.predict(testX) for model in members]
  yhats = array(yhats)
	# weighted sum across ensemble members
  summed = tensordot(yhats, weights, axes=((0),(0)))
	# argmax across classes
  result = argmax(summed, axis=1)
  return result
 
# Evaluate a specific number of members in an ensemble
def evaluate_ensemble(members, weights, testX, testy):
  # make prediction
  yhat = ensemble_predictions(members, weights, testX)
  # calculate accuracy
  score = accuracy_score(testy, yhat)
  print(score, weights)
  return score
 
# Normalize a vector to have unit norm
def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result
 
# Grid Search Weights
def grid_search(members, testX, testy):
	# define weights to consider
	w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
	best_score, best_weights = 0.0, None
	# iterate all possible combinations (cartesian product)
	for weights in product(w, repeat=len(members)):
		# skip if all weights are equal
		if len(set(weights)) == 1:
			continue
		# hack, normalize weight vector
		weights = normalize(weights)
		# evaluate weights
		score = evaluate_ensemble(members, weights, testX, testy)
		if score > best_score:
			best_score, best_weights = score, weights
			print('>%s %.3f' % (best_weights, best_score))
	return list(best_weights)

In [0]:
# Generate a batch of images for testing
members = [model1, model2]
test, test_labels = test_generator.next()

In [0]:
# For every model, evaluate each model to get the evaluation score
for i in range(1):
	_, test_acc = members[i].evaluate(test, test_labels, verbose=0)
	print('Model %d: %.3f' % (i+1, test_acc))

In [0]:
# Convert the test labels to required format for grid search
index = tf.argmax(test_labels, axis = 1)
session = tf.Session()
test_labels = session.run(index)

In [0]:
# Find the ideal weights which provides the most accurate outputs
weights = grid_search(members, test, test_labels)
score = evaluate_ensemble(members, weights, test, test_labels)
print('Grid Search Weights: %s, Score: %.3f' % (weights, score))